In [ ]:
import numpy as np
import pandas as pd
import os
from fastai.vision.all import *

In [ ]:
path = Path('../input/cassava-leaf-disease-classification/')
os.listdir(path)

In [ ]:
train = pd.read_csv(path / 'train.csv')
train.head()

In [ ]:
train['image_id'] = train['image_id'].map(lambda x : path /'train_images'/x )
train.head()

In [ ]:
def get_x(row): return row['image_id']
def get_y(row): return row['label']

db = DataBlock(blocks = (ImageBlock, CategoryBlock),
              get_x = get_x,
               get_y = get_y,
               splitter = RandomSplitter(valid_pct = 0.2),
               item_tfms = [Resize(128)],
               batch_tfms = [RandomResizedCropGPU(224), *aug_transforms(), 
                            Normalize.from_stats(*imagenet_stats)]            
              )

In [ ]:
dls = db.dataloaders(train, bs = 8)

In [ ]:
dls.show_batch(figsize = (12,12))

In [ ]:
if not os.path.exists('/root/.cache/torch/hub/checkpoints/'):
        os.makedirs('/root/.cache/torch/hub/checkpoints/')
!cp '../input/resnet152/resnet152.pth' '/root/.cache/torch/hub/checkpoints/resnet152-b121ed2d.pth'

In [ ]:
learn = cnn_learner(dls, resnet152, opt_func=ranger, loss_func=LabelSmoothingCrossEntropy(), metrics=accuracy)

In [ ]:
learn.fine_tune(6, freeze_epochs = 3)

In [ ]:
os.listdir(path)

In [ ]:
learn.save('resnet152')

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_confusion_matrix()

In [ ]:
sample = pd.read_csv(path/'sample_submission.csv')
sample

In [ ]:
_sample = sample.copy()
_sample['image_id'] = _sample['image_id'].map(lambda x:path/'test_images'/x)
test_dl = dls.test_dl(_sample)

In [ ]:
_sample.head()

In [ ]:
test_dl.show_batch()

In [ ]:
preds, _ = learn.tta(dl=test_dl, n=8, beta=0)

In [ ]:
sample['label'] = preds.argmax(dim=-1).numpy()

In [ ]:
sample.head()

In [ ]:
sample.to_csv('submission.csv',index=False)